##KNN

####Numpy (GPU accelerated)

In [1]:
import cupy as cp
import time
import os
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def knn_cupy(X_train, y_train, X_test, k=3):
    predictions = []
    for test_point in X_test:
        distances = cp.linalg.norm(X_train - test_point, axis=1)
        nearest = cp.argsort(distances)[:k]
        top_k_labels = y_train[nearest]
        predicted = cp.bincount(top_k_labels).argmax()
        predictions.append(predicted)
    return cp.array(predictions)


def estimate_bandwidth(X_train, X_test, time_seconds):
    bytes_read = X_train.nbytes + X_test.nbytes
    gb_read = bytes_read / 1e9
    return gb_read / time_seconds if time_seconds > 0 else 0

def main():
    # Load and split data (on CPU)
    X, y = load_digits(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Move data to GPU
    X_train_gpu = cp.asarray(X_train)
    X_test_gpu = cp.asarray(X_test)
    y_train_gpu = cp.asarray(y_train)

    # Track memory before
    mem_before, _ = cp.cuda.Device().mem_info

    # Time the KNN
    start = time.time()
    y_pred_gpu = knn_cupy(X_train_gpu, y_train_gpu, X_test_gpu, k=3)
    cp.cuda.Device().synchronize()
    end = time.time()

    # Convert predictions back to NumPy for accuracy check
    y_pred = cp.asnumpy(y_pred_gpu)
    acc = accuracy_score(y_test, y_pred)

    # Track memory after
    mem_after, _ = cp.cuda.Device().mem_info
    peak_used = (mem_before - mem_after) / 1e6  # in MB

    # Bandwidth
    exec_time = end - start
    bandwidth = estimate_bandwidth(X_train_gpu, X_test_gpu, exec_time)

    print(f"KNN-CuPy Accuracy: {acc:.2f}")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Peak GPU Memory Usage: {peak_used:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

if __name__ == "__main__":
    main()


KNN-CuPy Accuracy: 0.98
Execution Time: 9.4581 seconds
Peak GPU Memory Usage: 4.19 MB
Estimated Memory Bandwidth: 0.00 GB/s


####KNN for large data

In [2]:
import numpy as np
import cupy as cp
import time
import tracemalloc
import psutil
import os
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def knn_numpy(X_train, y_train, X_test, k=3):
    predictions = []
    for test_point in X_test:
        distances = np.linalg.norm(X_train - test_point, axis=1)
        nearest = np.argsort(distances)[:k]
        top_k_labels = y_train[nearest]
        predicted = np.bincount(top_k_labels).argmax()
        predictions.append(predicted)
    return np.array(predictions)

def knn_cupy(X_train, y_train, X_test, k=3):
    predictions = []
    for test_point in X_test:
        distances = cp.linalg.norm(X_train - test_point, axis=1)
        nearest = cp.argsort(distances)[:k]
        top_k_labels = y_train[nearest]
        predicted = cp.bincount(top_k_labels).argmax()
        predictions.append(predicted)
    return cp.array(predictions)

def estimate_bandwidth(X_train, X_test, time_seconds):
    total_bytes = X_train.nbytes + X_test.nbytes
    return (total_bytes / 1e9) / time_seconds if time_seconds > 0 else 0

def benchmark_knn_numpy(X_train, y_train, X_test, y_test):
    tracemalloc.start()
    start = time.time()
    y_pred = knn_numpy(X_train, y_train, X_test, k=3)
    end = time.time()
    _, peak = tracemalloc.get_traced_memory()
    peak_mb = peak / 1e6
    tracemalloc.stop()
    acc = accuracy_score(y_test, y_pred)
    bandwidth = estimate_bandwidth(X_train, X_test, end - start)
    print("\n KNN - NumPy (CPU)")
    print(f"Accuracy: {acc:.2f}")
    print(f"Execution Time: {end - start:.4f} seconds")
    print(f"Peak RAM Usage: {peak_mb:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

def benchmark_knn_cupy(X_train, y_train, X_test, y_test):
    # Move data to GPU
    X_train_gpu = cp.asarray(X_train)
    X_test_gpu = cp.asarray(X_test)
    y_train_gpu = cp.asarray(y_train)

    mem_before, _ = cp.cuda.Device().mem_info
    start = time.time()
    y_pred_gpu = knn_cupy(X_train_gpu, y_train_gpu, X_test_gpu, k=3)
    cp.cuda.Device().synchronize()
    end = time.time()
    mem_after, _ = cp.cuda.Device().mem_info
    peak_used = (mem_before - mem_after) / 1e6
    acc = accuracy_score(y_test, cp.asnumpy(y_pred_gpu))
    bandwidth = estimate_bandwidth(X_train_gpu, X_test_gpu, end - start)
    print("\n KNN - CuPy (GPU)")
    print(f"Accuracy: {acc:.2f}")
    print(f"Execution Time: {end - start:.4f} seconds")
    print(f"Peak GPU Memory Usage: {peak_used:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

def main():
    print(" Loading MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target.astype(int)

    # Normalize and sample a smaller subset for speed
    X = X / 255.0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, train_size=5000, random_state=42)

    benchmark_knn_numpy(X_train, y_train, X_test, y_test)
    benchmark_knn_cupy(X_train, y_train, X_test, y_test)

if __name__ == "__main__":
    main()


 Loading MNIST dataset...

 KNN - NumPy (CPU)
Accuracy: 0.93
Execution Time: 14.2214 seconds
Peak RAM Usage: 94.29 MB
Estimated Memory Bandwidth: 0.00 GB/s

 KNN - CuPy (GPU)
Accuracy: 0.93
Execution Time: 1.4518 seconds
Peak GPU Memory Usage: 31.46 MB
Estimated Memory Bandwidth: 0.03 GB/s


####pytorch for MNIST

In [3]:
import torch
import time
import tracemalloc
import psutil
import os
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def knn_torch(X_train, y_train, X_test, k=3):
    predictions = []
    for test_point in X_test:
        distances = torch.norm(X_train - test_point, dim=1)
        nearest = torch.topk(distances, k, largest=False).indices
        top_k_labels = y_train[nearest]
        predicted = torch.mode(top_k_labels).values.item()
        predictions.append(predicted)
    return torch.tensor(predictions)

def estimate_bandwidth(X_train, X_test, time_seconds):
    bytes_read = X_train.element_size() * (X_train.numel() + X_test.numel())
    gb_read = bytes_read / 1e9
    return gb_read / time_seconds if time_seconds > 1e-6 else 0.0

def benchmark_knn_torch(X_train_np, y_train_np, X_test_np, y_test_np, device):
    print(f"\n KNN - PyTorch ({device.upper()})")

    # Move to device
    X_train = torch.tensor(X_train_np, dtype=torch.float32, device=device)
    X_test = torch.tensor(X_test_np, dtype=torch.float32, device=device)
    y_train = torch.tensor(y_train_np, dtype=torch.int64, device=device)

    if device == 'cuda':
        torch.cuda.reset_peak_memory_stats()

    tracemalloc.start()
    start_time = time.time()
    y_pred = knn_torch(X_train, y_train, X_test, k=3)
    if device == 'cuda':
        torch.cuda.synchronize()
    end_time = time.time()

    # Accuracy
    y_pred_np = y_pred.cpu().numpy()
    acc = accuracy_score(y_test_np, y_pred_np)

    # Memory usage
    _, peak = tracemalloc.get_traced_memory()
    peak_mb = peak / 1e6

    # GPU memory (if applicable)
    gpu_mem = torch.cuda.max_memory_allocated() / 1e6 if device == 'cuda' else 0

    # Bandwidth
    exec_time = end_time - start_time
    bandwidth = estimate_bandwidth(X_train, X_test, exec_time)

    # Output
    print(f"Accuracy: {acc:.2f}")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Peak RAM Usage: {peak_mb:.2f} MB")
    if device == 'cuda':
        print(f"Peak GPU Memory Usage: {gpu_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

def main():
    print(" Loading MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target.astype(int)
    X = X.astype("float32") / 255.0

    # Use a subset for performance
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000, test_size=1000, random_state=42)

    benchmark_knn_torch(X_train, y_train, X_test, y_test, device='cpu')

    if torch.cuda.is_available():
        benchmark_knn_torch(X_train, y_train, X_test, y_test, device='cuda')

if __name__ == "__main__":
    main()


 Loading MNIST dataset...

 KNN - PyTorch (CPU)
Accuracy: 0.93
Execution Time: 3.3407 seconds
Peak RAM Usage: 0.16 MB
Estimated Memory Bandwidth: 0.01 GB/s

 KNN - PyTorch (CUDA)
Accuracy: 0.93
Execution Time: 0.6047 seconds
Peak RAM Usage: 0.17 MB
Peak GPU Memory Usage: 34.58 MB
Estimated Memory Bandwidth: 0.03 GB/s


####C++ (GPU accelerated)

In [4]:
%%writefile knn_gpu.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <cmath>
#include <map>
#include <algorithm>
#include <chrono>
#include <sys/resource.h>
#include <cuda_runtime.h>

using namespace std;
using namespace chrono;

#define BLOCK_SIZE 256

__global__ void compute_distances(const double* X_train, const double* x_test, double* distances, int num_train, int num_features) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_train) {
        double sum = 0.0;
        for (int j = 0; j < num_features; ++j) {
            double diff = X_train[idx * num_features + j] - x_test[j];
            sum += diff * diff;
        }
        distances[idx] = sqrt(sum);
    }
}

// Utility to load CSV
vector<vector<double>> load_csv_data(const string& filename, int rows, int cols) {
    vector<vector<double>> data(rows, vector<double>(cols));
    ifstream file(filename);
    string line;
    for (int i = 0; i < rows && getline(file, line); ++i) {
        stringstream ss(line);
        string val;
        for (int j = 0; j < cols && getline(ss, val, ','); ++j) {
            data[i][j] = stod(val);
        }
    }
    return data;
}

vector<int> load_csv_labels(const string& filename, int rows) {
    vector<int> labels(rows);
    ifstream file(filename);
    string line;
    for (int i = 0; i < rows && getline(file, line); ++i) {
        labels[i] = stoi(line);
    }
    return labels;
}

// KNN classifier using GPU distances
int knn_gpu_predict(const vector<vector<double>>& X_train, const vector<int>& y_train,
                    const vector<double>& x_test, int k, int num_features) {
    int num_train = X_train.size();

    double* d_X_train;
    double* d_x_test;
    double* d_distances;
    cudaMalloc(&d_X_train, num_train * num_features * sizeof(double));
    cudaMalloc(&d_x_test, num_features * sizeof(double));
    cudaMalloc(&d_distances, num_train * sizeof(double));

    // Flatten train
    vector<double> X_train_flat(num_train * num_features);
    for (int i = 0; i < num_train; ++i)
        for (int j = 0; j < num_features; ++j)
            X_train_flat[i * num_features + j] = X_train[i][j];

    cudaMemcpy(d_X_train, X_train_flat.data(), num_train * num_features * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_x_test, x_test.data(), num_features * sizeof(double), cudaMemcpyHostToDevice);

    int threads = BLOCK_SIZE;
    int blocks = (num_train + threads - 1) / threads;
    compute_distances<<<blocks, threads>>>(d_X_train, d_x_test, d_distances, num_train, num_features);
    cudaDeviceSynchronize();

    vector<double> distances(num_train);
    cudaMemcpy(distances.data(), d_distances, num_train * sizeof(double), cudaMemcpyDeviceToHost);

    vector<pair<double, int>> dist_label;
    for (int i = 0; i < num_train; ++i)
        dist_label.push_back({distances[i], y_train[i]});
    sort(dist_label.begin(), dist_label.end());

    map<int, int> count;
    for (int i = 0; i < k; ++i)
        count[dist_label[i].second]++;

    int best_label = -1, max_votes = -1;
    for (auto& [label, votes] : count) {
        if (votes > max_votes) {
            best_label = label;
            max_votes = votes;
        }
    }

    cudaFree(d_X_train);
    cudaFree(d_x_test);
    cudaFree(d_distances);
    return best_label;
}

void print_peak_ram_usage() {
    struct rusage usage;
    getrusage(RUSAGE_SELF, &usage);
    cout << "Peak RAM Usage: " << usage.ru_maxrss / 1024.0 << " MB" << endl;
}

int main() {
    int num_train = 1000;
    int num_test = 100;
    int num_features = 784;
    int k = 5;

    auto X_train = load_csv_data("X_train.csv", num_train, num_features);
    auto y_train = load_csv_labels("y_train.csv", num_train);
    auto X_test = load_csv_data("X_test.csv", num_test, num_features);
    auto y_test = load_csv_labels("y_test.csv", num_test);

    vector<int> y_pred;

    auto start = high_resolution_clock::now();

    for (int i = 0; i < num_test; ++i) {
        int pred = knn_gpu_predict(X_train, y_train, X_test[i], k, num_features);
        y_pred.push_back(pred);
    }

    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<milliseconds>(stop - start).count();
    int correct = 0;
    for (int i = 0; i < num_test; ++i) {
        if (y_pred[i] == y_test[i]) correct++;
    }

    cout << "KNN-CUDA Accuracy: " << static_cast<double>(correct) / num_test << endl;
    cout << "Execution Time: " << duration << " ms" << endl;
    print_peak_ram_usage();

    return 0;
}


Writing knn_gpu.cu


In [5]:
!nvcc -O2 knn_gpu.cu -o knn_gpu

In [6]:
!./knn_gpu

KNN-CUDA Accuracy: 1
Execution Time: 476 ms
Peak RAM Usage: 2190.63 MB


##Matrix multiplication

####Numpy (GPU accelerated)

In [7]:
import cupy as cp
import time

def estimate_bandwidth(A, B, time_seconds):
    bytes_read = A.nbytes + B.nbytes
    gb_read = bytes_read / 1e9
    return gb_read / time_seconds if time_seconds > 1e-6 else 0.0

def benchmark_dense(n=1024):
    print(" Dense Matrix Multiplication on GPU (A @ B)")

    A = cp.random.rand(n, n, dtype=cp.float32)
    B = cp.random.rand(n, n, dtype=cp.float32)

    cp.cuda.Device().synchronize()
    mem_before = cp.cuda.Device().mem_info[0]

    start_time = time.time()
    C = A @ B
    cp.cuda.Device().synchronize()
    end_time = time.time()

    mem_after = cp.cuda.Device().mem_info[0]
    peak_mem = (mem_before - mem_after) / 1e6  # MB

    exec_time = end_time - start_time
    bandwidth = estimate_bandwidth(A, B, exec_time)

    print(f"Shape: ({n}, {n}) x ({n}, {n})")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Approx. Peak GPU Memory Usage: {peak_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")


def benchmark_batched(batch_size=64, n=128):
    print("\n Batched Matrix Multiplication on GPU (A @ B)")

    A = cp.random.rand(batch_size, n, n, dtype=cp.float32)
    B = cp.random.rand(batch_size, n, n, dtype=cp.float32)

    cp.cuda.Device().synchronize()
    mem_before = cp.cuda.Device().mem_info[0]

    start_time = time.time()
    C = cp.matmul(A, B)
    cp.cuda.Device().synchronize()
    end_time = time.time()

    mem_after = cp.cuda.Device().mem_info[0]
    peak_mem = (mem_before - mem_after) / 1e6  # in MB

    exec_time = end_time - start_time
    bandwidth = estimate_bandwidth(A, B, exec_time)

    print(f"Shape: ({batch_size}, {n}, {n}) x ({batch_size}, {n}, {n})")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Approx. Peak GPU Memory Usage: {peak_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")


def main():
    benchmark_dense(n=1024)
    benchmark_batched(batch_size=64, n=128)

if __name__ == "__main__":
    main()


 Dense Matrix Multiplication on GPU (A @ B)
Shape: (1024, 1024) x (1024, 1024)
Execution Time: 0.1173 seconds
Approx. Peak GPU Memory Usage: 8.39 MB
Estimated Memory Bandwidth: 0.07 GB/s

 Batched Matrix Multiplication on GPU (A @ B)
Shape: (64, 128, 128) x (64, 128, 128)
Execution Time: 0.0016 seconds
Approx. Peak GPU Memory Usage: 0.00 MB
Estimated Memory Bandwidth: 5.30 GB/s



####PyTorch - GPU

In [8]:
import torch
import time

def estimate_bandwidth(A, B, time_seconds):
    bytes_read = A.element_size() * (A.numel() + B.numel())
    gb_read = bytes_read / 1e9
    return gb_read / time_seconds if time_seconds > 1e-6 else 0.0

def benchmark_dense(n=1024):
    print(" Dense Matrix Multiplication on GPU (A @ B)")

    A = torch.rand(n, n, dtype=torch.float32, device='cuda')
    B = torch.rand(n, n, dtype=torch.float32, device='cuda')

    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()
    C = A @ B
    torch.cuda.synchronize()
    end_time = time.time()

    exec_time = end_time - start_time
    peak_mem = torch.cuda.max_memory_allocated() / 1e6  # MB
    bandwidth = estimate_bandwidth(A, B, exec_time)

    print(f"Shape: ({n}, {n}) x ({n}, {n})")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Peak GPU Memory Usage: {peak_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

def benchmark_batched(batch_size=64, n=128):
    print("\n Batched Matrix Multiplication on GPU (A @ B)")

    A = torch.rand(batch_size, n, n, dtype=torch.float32, device='cuda')
    B = torch.rand(batch_size, n, n, dtype=torch.float32, device='cuda')

    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()
    C = torch.bmm(A, B)
    torch.cuda.synchronize()
    end_time = time.time()

    exec_time = end_time - start_time
    peak_mem = torch.cuda.max_memory_allocated() / 1e6  # MB
    bandwidth = estimate_bandwidth(A, B, exec_time)

    print(f"Shape: ({batch_size}, {n}, {n}) x ({batch_size}, {n}, {n})")
    print(f"Execution Time: {exec_time:.4f} seconds")
    print(f"Peak GPU Memory Usage: {peak_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")

def main():
    if not torch.cuda.is_available():
        print(" CUDA not available. Please run on a GPU-enabled machine.")
        return

    benchmark_dense(n=1024)
    benchmark_batched(batch_size=64, n=128)

if __name__ == "__main__":
    main()


 Dense Matrix Multiplication on GPU (A @ B)
Shape: (1024, 1024) x (1024, 1024)
Execution Time: 0.0127 seconds
Peak GPU Memory Usage: 21.10 MB
Estimated Memory Bandwidth: 0.66 GB/s

 Batched Matrix Multiplication on GPU (A @ B)
Shape: (64, 128, 128) x (64, 128, 128)
Execution Time: 0.0006 seconds
Peak GPU Memory Usage: 21.10 MB
Estimated Memory Bandwidth: 12.91 GB/s


####C++ (GPU accelerated)

In [9]:
%%writefile matrix_mul_dense_and_batch.cu
#include <iostream>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <chrono>

void checkCuda(cudaError_t result, const char* msg) {
    if (result != cudaSuccess) {
        std::cerr << "CUDA error in " << msg << ": " << cudaGetErrorString(result) << std::endl;
        exit(EXIT_FAILURE);
    }
}

void checkCublas(cublasStatus_t status, const char* msg) {
    if (status != CUBLAS_STATUS_SUCCESS) {
        std::cerr << "cuBLAS error in " << msg << ": " << status << std::endl;
        exit(EXIT_FAILURE);
    }
}

void run_dense(int N) {
    std::cout << "\n Dense Matrix Multiplication (CUDA kernel)" << std::endl;

    size_t size = N * N * sizeof(float);
    float *A, *B, *C;

    checkCuda(cudaMalloc(&A, size), "cudaMalloc A");
    checkCuda(cudaMalloc(&B, size), "cudaMalloc B");
    checkCuda(cudaMalloc(&C, size), "cudaMalloc C");

    float *h_A = new float[N * N];
    float *h_B = new float[N * N];
    for (int i = 0; i < N * N; ++i) {
        h_A[i] = static_cast<float>(rand()) / RAND_MAX;
        h_B[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    checkCuda(cudaMemcpy(A, h_A, size, cudaMemcpyHostToDevice), "copy A");
    checkCuda(cudaMemcpy(B, h_B, size, cudaMemcpyHostToDevice), "copy B");

    cublasHandle_t handle;
    checkCublas(cublasCreate(&handle), "cublasCreate");

    float alpha = 1.0f, beta = 0.0f;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    checkCublas(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N,
                            N, N, N,
                            &alpha,
                            A, N,
                            B, N,
                            &beta,
                            C, N),
                "cublasSgemm");

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    double seconds = ms / 1000.0;

    double bandwidth = (2.0 * size) / (seconds * 1e9);  // GB/s

    std::cout << "Matrix size: " << N << " x " << N << std::endl;
    std::cout << "Execution time: " << seconds << " seconds" << std::endl;
    std::cout << "Estimated bandwidth: " << bandwidth << " GB/s" << std::endl;

    cudaFree(A); cudaFree(B); cudaFree(C);
    delete[] h_A; delete[] h_B;
    cublasDestroy(handle);
}

void run_batched(int batch, int N) {
    std::cout << "\n Batched Matrix Multiplication (cuBLAS)" << std::endl;

    size_t size = batch * N * N * sizeof(float);
    float *A, *B, *C;

    checkCuda(cudaMalloc(&A, size), "malloc A");
    checkCuda(cudaMalloc(&B, size), "malloc B");
    checkCuda(cudaMalloc(&C, size), "malloc C");

    float *h_A = new float[batch * N * N];
    float *h_B = new float[batch * N * N];

    for (int i = 0; i < batch * N * N; ++i) {
        h_A[i] = static_cast<float>(rand()) / RAND_MAX;
        h_B[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    checkCuda(cudaMemcpy(A, h_A, size, cudaMemcpyHostToDevice), "copy A");
    checkCuda(cudaMemcpy(B, h_B, size, cudaMemcpyHostToDevice), "copy B");

    cublasHandle_t handle;
    checkCublas(cublasCreate(&handle), "create handle");

    float alpha = 1.0f, beta = 0.0f;

    long stride = N * N;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    checkCublas(cublasSgemmStridedBatched(handle,
        CUBLAS_OP_N, CUBLAS_OP_N,
        N, N, N,
        &alpha,
        A, N, stride,
        B, N, stride,
        &beta,
        C, N, stride,
        batch), "cublasSgemmStridedBatched");

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    double seconds = ms / 1000.0;

    double bandwidth = (2.0 * size) / (seconds * 1e9);

    std::cout << "Batch size: " << batch << ", Matrix size: " << N << " x " << N << std::endl;
    std::cout << "Execution time: " << seconds << " seconds" << std::endl;
    std::cout << "Estimated bandwidth: " << bandwidth << " GB/s" << std::endl;

    cudaFree(A); cudaFree(B); cudaFree(C);
    delete[] h_A; delete[] h_B;
    cublasDestroy(handle);
}

int main() {
    srand(42);
    run_dense(1024);
    run_batched(64, 128);
    return 0;
}


Writing matrix_mul_dense_and_batch.cu


In [10]:
!nvcc -lcublas -O2 matrix_mul_dense_and_batch.cu -o matmul_gpu

In [11]:
!./matmul_gpu


 Dense Matrix Multiplication (CUDA kernel)
Matrix size: 1024 x 1024
Execution time: 0.0652577 seconds
Estimated bandwidth: 0.128546 GB/s

 Batched Matrix Multiplication (cuBLAS)
Batch size: 64, Matrix size: 128 x 128
Execution time: 0.000675104 seconds
Estimated bandwidth: 12.4257 GB/s


##CNN

####Numpy - GPU

In [12]:
import cupy as cp
import time

def relu(x):
    return cp.maximum(0, x)

def maxpool(x, size=2, stride=2):
    n, c, h, w = x.shape
    out_h = (h - size) // stride + 1
    out_w = (w - size) // stride + 1
    out = cp.zeros((n, c, out_h, out_w), dtype=cp.float32)
    for i in range(out_h):
        for j in range(out_w):
            x_slice = x[:, :, i*stride:i*stride+size, j*stride:j*stride+size]
            out[:, :, i, j] = cp.max(x_slice, axis=(2, 3))
    return out

def conv2d(x, w, b, stride=1, padding=0):
    n, c_in, h, w_in = x.shape
    c_out, _, k, _ = w.shape
    h_out = (h + 2*padding - k) // stride + 1
    w_out = (w_in + 2*padding - k) // stride + 1

    x_padded = cp.pad(x, ((0,0), (0,0), (padding,padding), (padding,padding)), mode='constant')
    out = cp.zeros((n, c_out, h_out, w_out), dtype=cp.float32)

    for i in range(h_out):
        for j in range(w_out):
            x_slice = x_padded[:, :, i*stride:i*stride+k, j*stride:j*stride+k]
            for f in range(c_out):
                out[:, f, i, j] = cp.sum(x_slice * w[f, :, :, :], axis=(1,2,3))
    return out + b[None, :, None, None]

def flatten(x):
    return x.reshape(x.shape[0], -1)

def fully_connected(x, w, b):
    return x @ w.T + b

def estimate_bandwidth(arrays, time_seconds):
    total_bytes = sum([a.nbytes for a in arrays])
    return (total_bytes / 1e9) / time_seconds if time_seconds > 1e-6 else 0.0

def main():
    cp.random.seed(42)
    batch_size = 8
    x = cp.random.rand(batch_size, 1, 64, 64).astype(cp.float32)

    w1 = cp.random.rand(16, 1, 3, 3).astype(cp.float32)
    b1 = cp.random.rand(16).astype(cp.float32)

    w2 = cp.random.rand(32, 16, 3, 3).astype(cp.float32)
    b2 = cp.random.rand(32).astype(cp.float32)

    fc1_in = 32 * 16 * 16
    w_fc1 = cp.random.rand(128, fc1_in).astype(cp.float32)
    b_fc1 = cp.random.rand(128).astype(cp.float32)

    w_fc2 = cp.random.rand(10, 128).astype(cp.float32)
    b_fc2 = cp.random.rand(10).astype(cp.float32)

    cp.cuda.Device().synchronize()
    cp.cuda.runtime.deviceSynchronize()
    mem_before = cp.cuda.Device().mem_info[0]

    start_time = time.time()

    out = conv2d(x, w1, b1, stride=1, padding=1)
    out = relu(out)
    out = maxpool(out)

    out = conv2d(out, w2, b2, stride=1, padding=1)
    out = relu(out)
    out = maxpool(out)

    out = flatten(out)
    out = relu(fully_connected(out, w_fc1, b_fc1))
    out = fully_connected(out, w_fc2, b_fc2)

    cp.cuda.Device().synchronize()
    exec_time = time.time() - start_time
    mem_after = cp.cuda.Device().mem_info[0]
    peak_mem = (mem_before - mem_after) / 1e6

    bandwidth = estimate_bandwidth([x, w1, w2, out], exec_time)

    print(" Large CNN - CuPy (GPU)")
    print(f"Execution Time: {exec_time:.4f} sec")
    print(f"Approx. Peak GPU Memory Usage: {peak_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")
    print(f"Output shape: {out.shape}")

if __name__ == "__main__":
    main()


 Large CNN - CuPy (GPU)
Execution Time: 42.2536 sec
Approx. Peak GPU Memory Usage: 0.00 MB
Estimated Memory Bandwidth: 0.00 GB/s
Output shape: (8, 10)


####Pytorch - GPU accelerated

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)   # → (B, 16, 64, 64)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # → (B, 32, 32, 32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → (B, 16, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))  # → (B, 32, 16, 16)
        x = x.view(x.size(0), -1)             # → (B, 8192)
        x = F.relu(self.fc1(x))               # → (B, 128)
        x = self.fc2(x)                       # → (B, 10)
        return x

def estimate_bandwidth(tensors, exec_time):
    total_bytes = sum([t.element_size() * t.numel() for t in tensors])
    return (total_bytes / 1e9) / exec_time if exec_time > 1e-6 else 0.0

def main():
    torch.manual_seed(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f" Using device: {device}")

    batch_size = 8
    model = CNNModel().to(device)
    x = torch.randn(batch_size, 1, 64, 64, device=device)

    model.eval()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()
    with torch.no_grad():
        output = model(x)
    torch.cuda.synchronize()
    end_time = time.time()

    exec_time = end_time - start_time
    peak_gpu_mem = torch.cuda.max_memory_allocated(device=device) / 1e6  # MB
    bandwidth = estimate_bandwidth([x, output], exec_time)

    print(" CNN - PyTorch (GPU)")
    print(f"Execution Time: {exec_time:.4f} sec")
    print(f"Peak GPU Memory Usage: {peak_gpu_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")
    print(f"Output Shape: {output.shape}")

if __name__ == "__main__":
    main()


 Using device: cuda
 CNN - PyTorch (GPU)
Execution Time: 0.5216 sec
Peak GPU Memory Usage: 17.07 MB
Estimated Memory Bandwidth: 0.00 GB/s
Output Shape: torch.Size([8, 10])


**Increasing the batch size**

Model   --   BatchSize	 --   Winner

Small   	--     (< 32)	   --     CPU

Medium   --     (64–128)	 --    Depends

Large   -- 	   ( >128) --	        GPU

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → (B, 16, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))  # → (B, 32, 16, 16)
        x = x.view(x.size(0), -1)             # → (B, 8192)
        x = F.relu(self.fc1(x))               # → (B, 128)
        x = self.fc2(x)                       # → (B, 10)
        return x

def estimate_bandwidth(tensors, exec_time):
    total_bytes = sum([t.element_size() * t.numel() for t in tensors])
    return (total_bytes / 1e9) / exec_time if exec_time > 1e-6 else 0.0

def benchmark(device, batch_size):
    print(f"\n Running on: {device.upper()} with batch size {batch_size}")
    torch.manual_seed(42)
    model = CNNModel().to(device)
    x = torch.randn(batch_size, 1, 64, 64, device=device)

    model.eval()

    # Warm-up
    with torch.no_grad():
        _ = model(x)

    if device == 'cuda':
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.synchronize()

    start_time = time.time()
    with torch.no_grad():
        output = model(x)
    if device == 'cuda':
        torch.cuda.synchronize()
    end_time = time.time()

    exec_time = end_time - start_time
    mem = torch.cuda.max_memory_allocated() / 1e6 if device == 'cuda' else 0
    bandwidth = estimate_bandwidth([x, output], exec_time)

    print(f"Execution Time: {exec_time:.4f} sec")
    print(f"Peak {'GPU' if device=='cuda' else 'RAM'} Memory Usage: {mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")
    print(f"Output Shape: {output.shape}")

if __name__ == "__main__":
    batch_size = 128
    benchmark('cpu', batch_size)

    if torch.cuda.is_available():
        benchmark('cuda', batch_size)
    else:
        print(" GPU not available. Only CPU benchmark run.")



 Running on: CPU with batch size 128
Execution Time: 0.1356 sec
Peak RAM Memory Usage: 0.00 MB
Estimated Memory Bandwidth: 0.02 GB/s
Output Shape: torch.Size([128, 10])

 Running on: CUDA with batch size 128
Execution Time: 0.0029 sec
Peak GPU Memory Usage: 81.95 MB
Estimated Memory Bandwidth: 0.73 GB/s
Output Shape: torch.Size([128, 10])


####Pytorch with increased batch size

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

def estimate_bandwidth(tensors, time_sec):
    total_bytes = sum([t.element_size() * t.numel() for t in tensors])
    return (total_bytes / 1e9) / time_sec if time_sec > 0 else 0.0

def main():
    if not torch.cuda.is_available():
        print(" No CUDA device found.")
        return

    batch_size = 128
    device = "cuda"
    model = CNNModel().to(device)
    x = torch.randn(batch_size, 1, 64, 64, device=device)

    model.eval()

    with torch.no_grad():
        _ = model(x)  # warm-up

    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

    start = time.time()
    with torch.no_grad():
        output = model(x)
    torch.cuda.synchronize()
    end = time.time()

    exec_time = end - start
    peak_gpu_mem = torch.cuda.max_memory_allocated(device=device) / 1e6
    bandwidth = estimate_bandwidth([x, output], exec_time)

    print(" CNN - PyTorch (GPU)")
    print(f"Execution Time: {exec_time:.4f} sec")
    print(f"Peak GPU Memory Usage: {peak_gpu_mem:.2f} MB")
    print(f"Estimated Memory Bandwidth: {bandwidth:.2f} GB/s")
    print(f"Output Shape: {output.shape}")

if __name__ == "__main__":
    main()


 CNN - PyTorch (GPU)
Execution Time: 0.0035 sec
Peak GPU Memory Usage: 81.95 MB
Estimated Memory Bandwidth: 0.60 GB/s
Output Shape: torch.Size([128, 10])


####C++ with increased batch size

In [16]:
%%writefile cnn_forward_cuda.cu
// Paste the full code from the previous response here
#include <iostream>
#include <cuda_runtime.h>
#include <cmath>

#define IDX4(n, c, h, w, C, H, W) (((n)*(C)*(H)*(W)) + ((c)*(H)*(W)) + ((h)*(W)) + (w))

__global__ void conv2d(const float* x, const float* w, const float* b, float* out,
                       int N, int C_in, int H, int W, int C_out, int K, int H_out, int W_out, int padding) {
    int n = blockIdx.x;
    int f = threadIdx.x;

    for (int i = 0; i < H_out; ++i) {
        for (int j = 0; j < W_out; ++j) {
            float sum = 0.0f;
            for (int c = 0; c < C_in; ++c) {
                for (int ki = 0; ki < K; ++ki) {
                    for (int kj = 0; kj < K; ++kj) {
                        int row = i + ki - padding;
                        int col = j + kj - padding;
                        if (row >= 0 && row < H && col >= 0 && col < W) {
                            int idx_x = IDX4(n, c, row, col, C_in, H, W);
                            int idx_w = ((f * C_in + c) * K + ki) * K + kj;
                            sum += x[idx_x] * w[idx_w];
                        }
                    }
                }
            }
            int idx_out = IDX4(n, f, i, j, C_out, H_out, W_out);
            out[idx_out] = sum + b[f];
        }
    }
}

__global__ void relu(float* x, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size)
        x[i] = fmaxf(0.0f, x[i]);
}

__global__ void maxpool2d(const float* x, float* out, int N, int C, int H, int W, int pool_h, int pool_w, int stride) {
    int n = blockIdx.x;
    int c = threadIdx.x;
    int H_out = (H - pool_h) / stride + 1;
    int W_out = (W - pool_w) / stride + 1;

    for (int i = 0; i < H_out; ++i)
        for (int j = 0; j < W_out; ++j) {
            float max_val = -1e10;
            for (int ph = 0; ph < pool_h; ++ph)
                for (int pw = 0; pw < pool_w; ++pw) {
                    int h = i * stride + ph;
                    int w = j * stride + pw;
                    int idx = IDX4(n, c, h, w, C, H, W);
                    max_val = fmaxf(max_val, x[idx]);
                }
            int idx_out = IDX4(n, c, i, j, C, H_out, W_out);
            out[idx_out] = max_val;
        }
}

void fill_random(float* arr, int size, float scale = 0.01f) {
    for (int i = 0; i < size; ++i)
        arr[i] = scale * ((float)rand() / RAND_MAX);
}

int main() {
    const int N = 64, C = 1, H = 28, W = 28;
    const int K1 = 3, C1 = 8;
    const int K2 = 3, C2 = 16;
    const int padding = 1, pool = 2, stride = 2;

    int H1 = (H + 2 * padding - K1) + 1;
    int W1 = H1;
    int H1p = H1 / 2;
    int W1p = W1 / 2;

    int H2 = (H1p + 2 * padding - K2) + 1;
    int W2 = H2;
    int H2p = H2 / 2;
    int W2p = W2 / 2;

    size_t input_size = N * C * H * W * sizeof(float);
    float *h_input = new float[N * C * H * W];
    fill_random(h_input, N * C * H * W);

    float *d_input, *d_out1, *d_out2, *d_pool1, *d_pool2;
    cudaMalloc(&d_input, input_size);
    cudaMemcpy(d_input, h_input, input_size, cudaMemcpyHostToDevice);

    cudaMalloc(&d_out1, N * C1 * H1 * W1 * sizeof(float));
    cudaMalloc(&d_pool1, N * C1 * H1p * W1p * sizeof(float));
    cudaMalloc(&d_out2, N * C2 * H2 * W2 * sizeof(float));
    cudaMalloc(&d_pool2, N * C2 * H2p * W2p * sizeof(float));

    float *w1, *b1, *w2, *b2;
    cudaMallocManaged(&w1, C1 * C * K1 * K1 * sizeof(float));
    cudaMallocManaged(&b1, C1 * sizeof(float));
    cudaMallocManaged(&w2, C2 * C1 * K2 * K2 * sizeof(float));
    cudaMallocManaged(&b2, C2 * sizeof(float));
    fill_random(w1, C1 * C * K1 * K1);
    fill_random(w2, C2 * C1 * K2 * K2);
    fill_random(b1, C1);
    fill_random(b2, C2);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    conv2d<<<N, C1>>>(d_input, w1, b1, d_out1, N, C, H, W, C1, K1, H1, W1, padding);
    relu<<<(N*C1*H1*W1 + 255)/256, 256>>>(d_out1, N*C1*H1*W1);
    maxpool2d<<<N, C1>>>(d_out1, d_pool1, N, C1, H1, W1, pool, pool, stride);

    conv2d<<<N, C2>>>(d_pool1, w2, b2, d_out2, N, C1, H1p, W1p, C2, K2, H2, W2, padding);
    relu<<<(N*C2*H2*W2 + 255)/256, 256>>>(d_out2, N*C2*H2*W2);
    maxpool2d<<<N, C2>>>(d_out2, d_pool2, N, C2, H2, W2, pool, pool, stride);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms = 0;
    cudaEventElapsedTime(&ms, start, stop);

    std::cout << " CNN Forward Pass - CUDA (Batch size = " << N << ")\n";
    std::cout << "Execution Time: " << ms / 1000.0f << " seconds\n";
    std::cout << "Output shape: (" << N << ", " << C2 << ", " << H2p << ", " << W2p << ")\n";

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_out1); cudaFree(d_out2);
    cudaFree(d_pool1); cudaFree(d_pool2);
    cudaFree(w1); cudaFree(b1);
    cudaFree(w2); cudaFree(b2);
    delete[] h_input;

    return 0;
}


Writing cnn_forward_cuda.cu


In [17]:
!nvcc -O2 cnn_forward_cuda.cu -o cnn_cuda


In [18]:
!./cnn_cuda

 CNN Forward Pass - CUDA (Batch size = 64)
Execution Time: 0.0111805 seconds
Output shape: (64, 16, 7, 7)


##GRADIENT COMPUTATION

####Numpy

In [19]:
import cupy as cp
import time
import os

def quadratic_function(x):
    return cp.sum(x ** 2)

def compute_gradient_gpu(f, x, h=1e-5):
    grad = cp.zeros_like(x)
    for i in range(x.size):
        x_ph = x.copy()
        x_mh = x.copy()
        x_ph[i] += h
        x_mh[i] -= h
        grad[i] = (f(x_ph) - f(x_mh)) / (2 * h)
    return grad

def estimate_bandwidth(arrays, exec_time):
    total_bytes = sum([a.nbytes for a in arrays])
    return (total_bytes / 1e9) / exec_time if exec_time > 1e-6 else 0.0

def main():
    dim = 100_000
    num_iters = 3

    x = cp.random.rand(dim, dtype=cp.float64)

    cp.cuda.runtime.deviceSynchronize()
    cp.cuda.Device(0).synchronize()
    cp.cuda.runtime.deviceSynchronize()

    start = time.time()
    for _ in range(num_iters):
        grad = compute_gradient_gpu(quadratic_function, x.copy())
    cp.cuda.Device(0).synchronize()
    end = time.time()

    exec_time = end - start
    bandwidth = estimate_bandwidth([x, grad], exec_time)
    peak_gpu_mem = cp.get_default_memory_pool().used_bytes() / 1e6

    print(" GPU Gradient - CuPy")
    print(f"Input Dimension: {dim}")
    print(f"Iterations: {num_iters}")
    print(f"Execution Time: {exec_time:.4f} sec")
    print(f"Estimated Memory Bandwidth: {bandwidth:.4f} GB/s")
    print(f"Approx GPU Memory Usage: {peak_gpu_mem:.2f} MB")
    print(f"Gradient Norm: {cp.linalg.norm(grad):.4f}")

if __name__ == "__main__":
    main()


 GPU Gradient - CuPy
Input Dimension: 100000
Iterations: 3
Execution Time: 298.9970 sec
Estimated Memory Bandwidth: 0.0000 GB/s
Approx GPU Memory Usage: 1.60 MB
Gradient Norm: 364.3787


####PyTorch

In [20]:
import torch
import time

def quadratic_function(x):
    return torch.sum(x ** 2)

def compute_gradient_gpu(f, x, h=1e-5):
    grad = torch.zeros_like(x)
    for i in range(x.size(0)):
        x_ph = x.clone()
        x_mh = x.clone()
        x_ph[i] += h
        x_mh[i] -= h
        grad[i] = (f(x_ph) - f(x_mh)) / (2 * h)
    return grad

def estimate_bandwidth(tensors, exec_time):
    total_bytes = sum([t.element_size() * t.numel() for t in tensors])
    return (total_bytes / 1e9) / exec_time if exec_time > 1e-6 else 0.0

def main():
    dim = 100_000
    num_iters = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x = torch.rand(dim, dtype=torch.float64, device=device)

    # Warm-up
    torch.cuda.synchronize()
    _ = compute_gradient_gpu(quadratic_function, x.clone())

    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    start = time.time()

    for _ in range(num_iters):
        grad = compute_gradient_gpu(quadratic_function, x.clone())

    torch.cuda.synchronize()
    end = time.time()

    exec_time = end - start
    bandwidth = estimate_bandwidth([x, grad], exec_time)
    peak_gpu_mem = torch.cuda.max_memory_allocated() / 1e6  # in MB

    print(" Gradient Computation - PyTorch (GPU)")
    print(f"Input Dimension     : {dim}")
    print(f"Iterations          : {num_iters}")
    print(f"Execution Time      : {exec_time:.4f} sec")
    print(f"Peak GPU Memory     : {peak_gpu_mem:.2f} MB")
    print(f"Estimated Bandwidth : {bandwidth:.4f} GB/s")
    print(f"Gradient Norm       : {torch.linalg.norm(grad).item():.4f}")

if __name__ == "__main__":
    main()


 Gradient Computation - PyTorch (GPU)
Input Dimension     : 100000
Iterations          : 3
Execution Time      : 53.3157 sec
Peak GPU Memory     : 14.12 MB
Estimated Bandwidth : 0.0000 GB/s
Gradient Norm       : 364.8179


####C++

In [21]:
%%writefile gradient_gpu.cu
//  Paste the full CUDA code here (see above)
#include <iostream>
#include <cmath>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;
using namespace chrono;

__global__ void compute_gradient(double* x, double* grad, int dim, double h) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < dim) {
        double orig = x[i];
        double x_ph = orig + h;
        double x_mh = orig - h;
        grad[i] = ((x_ph * x_ph) - (x_mh * x_mh)) / (2.0 * h);  // since f(x) = sum(x_i^2)
    }
}

double norm(const vector<double>& v) {
    double sum_sq = 0.0;
    for (double val : v)
        sum_sq += val * val;
    return sqrt(sum_sq);
}

int main() {
    int dim = 100000;
    int iters = 3;
    double h = 1e-5;

    vector<double> x(dim);
    for (int i = 0; i < dim; ++i)
        x[i] = static_cast<double>(rand()) / RAND_MAX;

    double *d_x, *d_grad;
    cudaMalloc(&d_x, dim * sizeof(double));
    cudaMalloc(&d_grad, dim * sizeof(double));
    cudaMemcpy(d_x, x.data(), dim * sizeof(double), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (dim + blockSize - 1) / blockSize;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    for (int i = 0; i < iters; ++i)
        compute_gradient<<<gridSize, blockSize>>>(d_x, d_grad, dim, h);

    cudaDeviceSynchronize();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms = 0.0f;
    cudaEventElapsedTime(&ms, start, stop);
    double seconds = ms / 1000.0;

    vector<double> grad(dim);
    cudaMemcpy(grad.data(), d_grad, dim * sizeof(double), cudaMemcpyDeviceToHost);

    double total_bytes = 2.0 * dim * sizeof(double) * iters;
    double bandwidth = (total_bytes / 1e9) / seconds;

    cout << " Gradient Computation - CUDA (GPU)" << endl;
    cout << "Input Dimension     : " << dim << endl;
    cout << "Iterations          : " << iters << endl;
    cout << "Execution Time      : " << seconds << " sec" << endl;
    cout << "Estimated Bandwidth : " << bandwidth << " GB/s" << endl;
    cout << "Gradient Norm       : " << norm(grad) << endl;

    cudaFree(d_x);
    cudaFree(d_grad);

    return 0;
}


Writing gradient_gpu.cu


In [22]:
!nvcc -O2 gradient_gpu.cu -o gradient_gpu

In [23]:
!./gradient_gpu

 Gradient Computation - CUDA (GPU)
Input Dimension     : 100000
Iterations          : 3
Execution Time      : 0.0130922 sec
Estimated Bandwidth : 0.36663 GB/s
Gradient Norm       : 0
